## Thursday, April 11, 2024

mamba activate langchain3

Always start with the LMStudio test.

In [1]:
# Example: reuse your existing OpenAI setup
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

completion = client.chat.completions.create(
  model="TheBloke/Mistral-7B-Instruct-v0.2-GGUF/mistral-7b-instruct-v0.2.Q8_0.gguf",
  messages=[
    {"role": "system", "content": "Always answer in rhymes."},
    {"role": "user", "content": "Introduce yourself."}
  ],
  temperature=0.7,
)

print(completion.choices[0].message)

ChatCompletionMessage(content=" In verse and prose, I'm here to please,\nA wordsmith with a rhyme's decree.\nI'll spin a yarn or tell a tale,\nMy name is Rhyme, I'm never stale.\n\nSo greetings to you, esteemed friend,\nMay our time together never end.\nLet's journey through this poetic land,\nHand in hand, with words as our command.", role='assistant', function_call=None, tool_calls=None)


Load the OpenAI API key in case we want to experiment with it in this notebook.

In [5]:
import os
import openai

OPENAI_API_KEY_ = os.environ['OPENAI_API_KEY_'] 
openai.api_key = OPENAI_API_KEY_

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY_

LLMs

In [2]:
from langchain.llms import OpenAI

# yes, we replicate the code from above to use the LangChain version of the OpenAI api ... 
llm = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

/home/rob/miniforge3/envs/langchain3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [3]:
response = llm.invoke("List the seven wonders of the world.")
print(response)



The Seven Wonders of the Ancient World are:

1. The Great Pyramid of Giza, Egypt (built around 2580 BCE)
2. The Hanging Gardens of Babylon, Mesopotamia (circa 600 BCE)
3. The Temple of Artemis at Ephesus, Turkey (built around 550 BCE)
4. The Statue of Zeus at Olympia, Greece (built around 435 BCE)
5. The Mausoleum at Halicarnassus, Turkey (built between 351 and 344 BCE)
6. The Colossus of Rhodes, Greece (built in 280 BCE)
7. The Lighthouse of Alexandria, Egypt (built around 280 BCE)

These seven wonders are among the most famous structures from ancient times and have inspired awe and wonder for thousands of years. However, it's important to note that there is some debate about their exact locations and historical accuracy. Some scholars believe that the list may have been created as a rhetorical device rather than


In [4]:
for chunk in llm.stream("Where were the 2012 Olympics held?"):
    print(chunk, end="", flush=True)


The 2012 Summer Olympics, officially known as the Games of the XXX Olympiad, were held in London, United Kingdom. The Olympic and Paralympic Games took place between 27 July and 12 September 2012. This marked the third time that London had hosted the Summer Olympics, having previously done so in 1908 and 1948. The games were held at various venues across the city, with the Olympic Stadium located in Stratford, Newham being the main stadium for the opening and closing ceremonies as well as the track and field events.

Chat Models

In [5]:
from langchain.chat_models import ChatOpenAI
chat = ChatOpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

/home/rob/miniforge3/envs/langchain3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [6]:
from langchain.schema.messages import HumanMessage, SystemMessage
messages = [
    SystemMessage(content="You are Micheal Jordan."),
    HumanMessage(content="Which shoe manufacturer are you associated with?"),
]
response = chat.invoke(messages)
print(response.content)

 I am most famously associated with the shoe manufacturer, Nike. We collaborated on the creation of the Air Jordan brand and released several iconic sneaker models during my professional basketball career.


Prompts

In [7]:
from langchain.prompts import PromptTemplate

In [8]:
# Simple prompt with placeholders
prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)

In [9]:
# Filling placeholders to create a prompt
filled_prompt = prompt_template.format(adjective="funny", content="robots")
print(filled_prompt)

Tell me a funny joke about robots.


In [10]:
from langchain.prompts import ChatPromptTemplate

# Defining a chat prompt with various roles
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)

# Formatting the chat prompt
formatted_messages = chat_template.format_messages(name="Bob", user_input="What is your name?")
for message in formatted_messages:
    print(message)

content='You are a helpful AI bot. Your name is Bob.'
content='Hello, how are you doing?'
content="I'm doing well, thanks!"
content='What is your name?'


PydanticOutputParser

In [11]:
from typing import List
from langchain.llms import OpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.pydantic_v1 import BaseModel, Field, validator

In [14]:
# Initialize the language model ... keep this here to optionally run the code below through OpenAI's API
# model = OpenAI(model_name="text-davinci-003", temperature=0.0)
# 'The model `text-davinci-003` has been deprecated ... 'gpt-3.5-turbo-instruct'

model = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.0)


In [12]:
# Initialize the language model
# model = OpenAI(model_name="text-davinci-003", temperature=0.0)
model = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio", temperature=0.0)

In [13]:
# Define your desired data structure using Pydantic
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field

In [14]:
# Set up a PydanticOutputParser
parser = PydanticOutputParser(pydantic_object=Joke)

In [15]:
# Create a prompt with format instructions
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)


In [16]:
# Define a query to prompt the language model
query = "Tell me a joke."

The next cell works if we use OpenAI ...

In [19]:
# OpenAI response ...
# Combine prompt, model, and parser to get structured output
prompt_and_model = prompt | model
output = prompt_and_model.invoke({"query": query})

The next call fails if we are using LMStudio ...  using the 'LoneStriker/gemma-1.1-7b-it-GGUF/gemma-1.1-7b-it-Q8_0.gguf' model. 

But if we use 'TheBloke/Mistral-7B-Instruct-v0.2-GGUF/mistral-7b-instruct-v0.2.Q4_K_S.gguf' it works! Or if we use the 'TheBloke/Mistral-7B-Instruct-v0.2-GGUF/mistral-7b-instruct-v0.2.Q8_0.gguf'model.

In [17]:
# LMStudio response ...
# Combine prompt, model, and parser to get structured output
prompt_and_model = prompt | model
output = prompt_and_model.invoke({"query": query})

In [18]:
# Parse the output using the parser
parsed_result = parser.invoke(output)

In [19]:
# The result is a structured object
print(parsed_result)

setup='Why did the tomato turn red?' punchline='Because it saw the salad dressing!'


SimplejsonOutputParser

This next cell returns results if we use the 'TheBloke/Mistral-7B-Instruct-v0.2-GGUF/mistral-7b-instruct-v0.2.Q4_K_S.gguf' model, but if we use 'TheBloke/Mistral-7B-Instruct-v0.2-GGUF/mistral-7b-instruct-v0.2.Q8_0.gguf' then the list returned is empty.

It also works if we use 'TheBloke/OpenHermes-2.5-Mistral-7B-GGUF/openhermes-2.5-mistral-7b.Q8_0.gguf'

In [25]:
from langchain.output_parsers.json import SimpleJsonOutputParser

# Create a JSON prompt
json_prompt = PromptTemplate.from_template(
    "Return a JSON object with `birthdate` and `birthplace` key that answers the following question: {question}"
)

# Initialize the JSON parser
json_parser = SimpleJsonOutputParser()

# Create a chain with the prompt, model, and parser
json_chain = json_prompt | model | json_parser

# Stream through the results
result_list = list(json_chain.stream({"question": "When and where was Elon Musk born?"}))

# The result is a list of JSON-like dictionaries
print(result_list)

[{}, {'birthdate': ''}, {'birthdate': 'J'}, {'birthdate': 'June'}, {'birthdate': 'June 2'}, {'birthdate': 'June 28'}, {'birthdate': 'June 28,'}, {'birthdate': 'June 28, 1'}, {'birthdate': 'June 28, 19'}, {'birthdate': 'June 28, 197'}, {'birthdate': 'June 28, 1971'}, {'birthdate': 'June 28, 1971', 'birthplace': ''}, {'birthdate': 'June 28, 1971', 'birthplace': 'P'}, {'birthdate': 'June 28, 1971', 'birthplace': 'Pret'}, {'birthdate': 'June 28, 1971', 'birthplace': 'Pretoria'}, {'birthdate': 'June 28, 1971', 'birthplace': 'Pretoria,'}, {'birthdate': 'June 28, 1971', 'birthplace': 'Pretoria, South'}, {'birthdate': 'June 28, 1971', 'birthplace': 'Pretoria, South Africa'}]


CommaSepartedListOutputParser

In [26]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

# Initialize the parser
output_parser = CommaSeparatedListOutputParser()

# Create format instructions
format_instructions = output_parser.get_format_instructions()

# Create a prompt to request a list
prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)

# Define a query to prompt the model
query = "English Premier League Teams"

# Generate the output
output = model(prompt.format(subject=query))

# Parse the output using the parser
parsed_result = output_parser.parse(output)

# The result is a list of items
print(parsed_result)

/home/rob/miniforge3/envs/langchain3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


['Manchester United', 'Manchester City', 'Liverpool', 'Chelsea', 'Arsenal']


DatetimeOutputParser

"TheBloke/Nous-Hermes-2-SOLAR-10.7B-GGUF/nous-hermes-2-solar-10.7b.Q8_0.gguf" worked! Once! Then when I re-ran this, it failed!!

"TheBloke/Mistral-7B-Instruct-v0.2-GGUF/mistral-7b-instruct-v0.2.Q8_0.gguf" fails!

"TheBloke/dolphin-2.6-mistral-7B-GGUF/dolphin-2.6-mistral-7b.Q8_0.gguf" fails!



In [42]:
from langchain.prompts import PromptTemplate
from langchain.output_parsers import DatetimeOutputParser
from langchain.chains import LLMChain
from langchain.llms import OpenAI

# Initialize the DatetimeOutputParser
output_parser = DatetimeOutputParser()

# Create a prompt with format instructions
template = """
Answer the user's question:
{question}
{format_instructions}
"""

prompt = PromptTemplate.from_template(
    template,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

# Create a chain with the prompt and language model
# chain = LLMChain(prompt=prompt, llm=OpenAI())
chain = LLMChain(prompt=prompt, llm=OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio", temperature=0.0))

# Define a query to prompt the model
query = "when did Neil Armstrong land on the moon in terms of GMT?"

# Run the chain
output = chain.run(query)



... response from the model that worked ... the 'text' field MUST be in the correct format for this to work!!

[2024-04-11 14:48:40.348] [INFO] Generated prediction: {
  "id": "cmpl-lor3fkoirlmdohkhbguac8",
  "object": "text_completion",
  "created": 1712861319,
  "model": "TheBloke/Nous-Hermes-2-SOLAR-10.7B-GGUF/nous-hermes-2-solar-10.7b.Q8_0.gguf",
  "choices": [
    {
      "index": 0,
      "text": "\n2022-07-20T12:17:39.156000Z",
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 28,
    "completion_tokens": 28,
    "total_tokens": 56
  }
}

In [43]:
output

'\n2022-07-20T12:17:39.156000Z'

In [44]:
# Parse the output using the datetime parser
parsed_result = output_parser.parse(output)

# The result is a datetime object
print(parsed_result)

2022-07-20 12:17:39.156000
